<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
# from plan import plan; plan("compléments", "type hints")

# type hints

* **suggestions** de typage

### motivations

* *duck typing* : pratique mais a des limitations
* introduire un mécanisme **optionnel** pour améliorer la situation
  * meilleure documentation / abaisser barrière d'entrée
  * analyse statique: trouver les bugs plus tôt
  * (performances)

# histoire

* commencé au travers du projet [mypy](http://mypy.readthedocs.io/en/latest/index.html), par Jukka Lehtosalo
* PEPs en vigueur
  * [PEP-484](https://www.python.org/dev/peps/pep-0484/) "Type hints"
  * [PEP-483](https://www.python.org/dev/peps/pep-0483/) "The theory of type hints"
* progressivement intégré à python
* module `typing` disponible depuis 3.5

# un exemple

pour insister sur le caractère **optionnel**:

In [ ]:
# on ajoute des annotations à l'objet 'ajouter'
def ajouter(x: int, y:int) -> int:
    return x + y

In [ ]:
# mais à run-time celles-ci sont ignorées !
ajouter('abc', 'def')

# à quoi ça sert alors ?

## utiliser un outil externe à la *pylint*

In [ ]:
!cat samples/types01.py

In [ ]:
!mypy samples/types01.py

# à quoi ça sert alors ?

### meilleure documentation !

* moins de temps perdu à deviner les présupposés sur les arguments

### comment définir un type

* les classes builtin `str`, `dict` etc..
* le module `typing` introduit des concepts additionnels
  * qui servent à étendre le spectre
  * comme `Iterable`, `Callable`, ...
  * mais aussi la notion de type abstrait avec `Generic` et `TypeVar`
  * nous allons voir tout ceci sur quelques exemples

## aliases

In [ ]:
# pour définir un alias pour une classe native
# une affectation suffit

Url = str

def retrieve_url(url: Url, count: int) -> bool:
    #
    return True

## classes

* un objet classe défini avec `class`
* peut être utilisé bien évidemment aussi

In [ ]:
class Foo: 
    pass

def link_foos(foo1: Foo, foo2: Foo) -> None:
    pass

## le module `typing`

* des **constructeurs de type**
* permettent de fabriquer des types plus élaborés
* ils sont définis **dans le module `typing`**
* et ont un nom en `FonteMixte`

## `List` et `Tuple`

* pour commencer: `List` (et `Tuple`)
* qui permettent de décrire le type des composants d'une liste
* remarquer l'usage des `[]` pour la composition de types

In [ ]:
from typing import List, Tuple

# une liste ayant un nombre quelconque d'éléments
# on décrit un objet `List` avec un seul type

# ce type décrit une liste d'objets tous de type str
Labels = List[str]

In [ ]:
# un tuple est non mutable, il semble logique de dire
# combien il doit avoir de composants et de quels types

# ce type décrit un tuple qui contient un entier et un flottant
Name = str
Age = int
Phone = str

Employee = Tuple[Name, Age, Phone]

In [ ]:
# Employee peut être mentionné 
# dans les type hints 
type(Employee)

In [ ]:
# mais ce n'est pas 
# une usine à objets
try: 
    Employee()
except Exception as exc:
    print(f"OOPS - {type(exc)}")

In [ ]:
# de manière similaire
# ne pas confondre tuple et Tuple !
Tuple is tuple

## `Dict` et `Set`

In [ ]:
# c'est pareil avec `Dict` et `Set`
from typing import Dict, Set

# Dict est construit avec deux types
NameHash = Dict[Name, Employee]

# et Set avec un seul
PhoneSet = Set[Phone]

In [ ]:
!cat samples/types02.py

In [ ]:
!mypy samples/types02.py

## `Iterable`, `Iterator`, `Sequence`

* ça devient intéressant de **formaliser**
* le vocabulaire qui est souvent un peu approximatif
* par exemple:

In [ ]:
from typing import Iterable, Iterator, Sequence

Iterable[int]
Iterator[Tuple[int, float, str]]
Sequence[List[float]]

## `NewType`

* Une méthode plus propre pour définir un alias

In [ ]:
from typing import NewType

UserId = NewType('UserId', int)
some_id = UserId(524313)

de cette façon on peut être plus strict

In [ ]:
def get_user_name(user_id: UserId) -> str:
    ...

# typecheck OK
user_a = get_user_name(UserId(42351))

# typecheck KO; un entier n'est pas un UserId
user_b = get_user_name(-1)

## autres constructeurs en vrac

* `Any` peut être n'importe quoi
* `Union` lorsqu'on accepte plusieurs types
* `Callable` pour les objets, ahem, callables
* `Hashable` ce qui peut être utilisé comme clé d'un dictionnaire
* `TypeVar` pour manipuler des types génériques (à la template C++)
  * grâce auxquelles on peut implémenter des classes génériques

# conclusion

* peut-être pas totalement stable encore
  * mais gagne petit à petit en popularité
  * il faut au moins savoir le lire !
* surtout en termes d'usage
  * tendance à aller vers un **type-checker *runtime* optionnel**
  * mais de nombreux points restent incertains 
* **par contre** c'est clairement la voie à suivre 
  * langage de type complet
  * pour enrichir la documentation et l'utilisabilité du code
  * notamment intégré dans les outils de doc (e.g. sphinx)

# partie optionnelle

## ATTENTION avec `isinstance`

* on serait tenté d'utiliser `isinstance`/`issubclass` avec les types
* il **ne faut pas le faire**
* je vous renvoie [à ce post](https://github.com/python/typing/issues/136) 
* il est suggéré de disposer d'une **autre** builtin que `isinstance`
  * pour vérifier si une variable est *acceptable* pour un **type**
  * alors que `isinstance` se base uniquement sur l'héritage de **classes**

In [ ]:
# ceci déclenche un TypeError
try:
    isinstance( {'0123456789', 98765432}, PhoneSet)
except Exception as exc:
    print(f"{type(exc)} {exc}")